In [32]:
import numpy as np

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import BasicAer
from qiskit.compiler import transpile
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.quantum_info import process_fidelity
from qiskit.providers.aer import QasmSimulator

In [41]:
backend = BasicAer.get_backend("qasm_simulator")

In [66]:
h_zz = -1
h_z = 1
h_x = 1

In [99]:
def state():
    circuit = QuantumCircuit(3, 3)
    circuit.h(0)
    circuit.cx(0, 1)
    circuit.z(1)
    circuit.x(2)
    circuit.cx(2, 0)
    
    return circuit

In [88]:
def evaluate(circuit, shots):
    circuit.measure_all() #different from circuit.measure([0,1], [0,1])?
    compiled = transpile(circuit, backend)
    job = backend.run(compiled, shots=shots)
    result = job.result()
    counts = result.get_counts(compiled)
    
    return counts

In [57]:
circuit = state()
evaluate(circuit, 1000)

{'00': 248, '01': 245, '10': 250, '11': 257}

In [60]:
def bit_parity(substr):
    parity = 1
    for char in substr:
        if char == '1':
            parity *= -1
    return parity

In [100]:
def get_EV(circuit, shots):
    print(circuit.draw())
    
    counts = evaluate(circuit, shots)
    op_z = 0
    op_zz = 0
    for bitstr in counts:
        p = counts[bitstr]/shots #probability of getting bitstr
        for index in range(len(bitstr)-1, 0, -1): #single values
            substr = bitstr[index] #one number, 0 or 1
            parity = bit_parity(substr)
            op_z += p * parity
        for index in range(len(bitstr)-1, 1, -1): #goes to 1 to avoid going under array index 0
            substr = bitstr[index-1:index+1] #string of two numbers
            parity = bit_parity(substr)
            op_zz += p * parity
    
    for i in range(circuit.num_qubits):
        circuit.h(i)
    counts = evaluate(circuit, shots)
    op_x = 0
    for bitstr in counts:
        p = counts[bitstr]/shots
        for index in range(len(bitstr)-1, 0, -1):
            substr = bitstr[index]
            parity = bit_parity(substr)
            op_x += p * parity
    
    print(op_zz)
    print(op_z)
    print(op_x)
    
    expected_value = h_zz * op_zz + h_z * op_z + h_x * op_x
    return expected_value

In [104]:
get_EV(state(), 1000)

     ┌───┐          ┌───┐
q_0: ┤ H ├──■───────┤ X ├
     └───┘┌─┴─┐┌───┐└─┬─┘
q_1: ─────┤ X ├┤ Z ├──┼──
     ┌───┐└───┘└───┘  │  
q_2: ┤ X ├────────────■──
     └───┘               
c: 3/════════════════════
                         
1.992
4.0
3.904000000000002


5.912000000000003